Auother: Tal Dvir

[The API site](https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to)

In [ ]:
!pip install mplfinance


In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1IbcmIhLAZmc5N6OyuRS9zHs8No9ma7tc' -O config.txt

--2024-03-12 19:56:45--  https://drive.google.com/uc?export=download&id=1IbcmIhLAZmc5N6OyuRS9zHs8No9ma7tc
Resolving drive.google.com (drive.google.com)... 142.250.128.100, 142.250.128.113, 142.250.128.101, ...
Connecting to drive.google.com (drive.google.com)|142.250.128.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1IbcmIhLAZmc5N6OyuRS9zHs8No9ma7tc&export=download [following]
--2024-03-12 19:56:45--  https://drive.usercontent.google.com/download?id=1IbcmIhLAZmc5N6OyuRS9zHs8No9ma7tc&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.120.132, 2607:f8b0:4001:c2e::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.120.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81 [application/octet-stream]
Saving to: ‘config.txt’

config.txt          100%[===================>]      81  --.-KB/s    in 0s 

In [ ]:
import requests
import datetime
import matplotlib.pyplot as plt
import yfinance as yf #(for the dataset)
import mplfinance as mpf
import yfinance as yf #(for the dataset)


In [ ]:


# Function to fetch data from the stock API
def get_stock_data(symbol, key, to, from_):

    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{from_}/{to}?apiKey={key}'

    try:
      response = requests.get(url)
      response.raise_for_status()  #Raise an exception for non-200 status codes

      data =  response.json()
      return data

    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

# Function to load configurations from a text file
def load_config(filename):


    config = {}
    with open(filename, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # print("print line : " + line)
            key, value = line.strip().split('=')
            config[key] = value

        print(f"print config: {config}")
    return config

def convert_unix_msec_to_datetime(unix_msec_timestamp):

    unix_sec_timestamp = unix_msec_timestamp / 1000
    # Create a datetime object from the timestamp
    dt_object = datetime.datetime.utcfromtimestamp(unix_sec_timestamp)
    return dt_object


# Function to write statistics to file. from a text file
def write_output(filename, data):

    with open(filename, 'w') as file:
      for line in data:
          file.write(str(line) +"\n")


In [ ]:

def get_input(symbol, from_, to):

    # assumtion, the API doesn't support today's date

    # Getting user's input request
    input_symbol = input("Enter the stock symbol(for defualt press enter): \n").upper() #symbols must be uppercase
    symbol = input_symbol if input_symbol else symbol

    input_from = input("Insert the start date in format '%YYYY-%mm-%dd'(for default press enter): \n")
    input_to = input("Insert the end date in format '%YYYY-%mm-%dd' (for default press enter): \n")


    # Convert the dates from string to datetime.datetime object to handle unexpected inputs like "asdfa" or 14sdfs(int) or unvalide date
    # The .date() is to omite the default 00:00:00 (midnight) that comes with the datetime object
    try:
      input_to = datetime.datetime.strptime(input_to, "%Y-%m-%d").date() if input_to else None
      input_from = datetime.datetime.strptime(input_from, "%Y-%m-%d").date() if input_from else None

      # print(input_to ,input_from)
    except Exception as e:
      print(f"Error: {e}")

    # convert for the option to use  '<' and not get unsupported opration error
    from_datetime =  datetime.datetime.strptime(from_ , "%Y-%m-%d").date()
    to_datetime = datetime.datetime.strptime(to , "%Y-%m-%d").date()

    # checking edge cases to validate the to and from dates
    if input_to and input_from and input_to < input_from:
        raise Exception("End date cannot be before start date.")

    elif  input_to and  input_to < from_datetime:
        raise Exception("End date cannot be before default start date.")

    elif input_from and to_datetime < input_from:
        raise Exception("From date cannot be After default end date.")
    else:
      to = input_to if input_to else to
      from_ = input_from if input_from else from_


    return (symbol, from_, to) # tuple


In [ ]:

def plot_line(commerce_dates, open_close, symbol):
    print("A line chart is a way of visually representing an asset's price history using a single, continuous line. \n")
    dates = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in commerce_dates]
    close_prices = [item[1] for item in open_close]

    percentage = (close_prices[-1] / close_prices[0]) * 100 -100  #geeting the defference

    color = 'g' if close_prices[0] < close_prices[-1] else 'r'
    if close_prices[0] == close_prices[-1]:
      color = 'gray'
      percentage = 0

    percentage = f"Current Price: {close_prices[-1]},  {percentage:.2f}%"


    plt.figure(figsize=(12, 6), dpi=500)
    plt.plot(dates, close_prices, marker=',')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'{symbol}')
    plt.suptitle(percentage, fontsize=12, fontweight='bold', color=color)  # Adding the colored subtitle
    plt.tight_layout()
    plt.xticks(dates[::10], rotation=45)  # Adjust the step size as needed, here every 10th date is shown with a rotation of 45 degrees

    plt.show()



def plot_candle(symbol, from_, to):
    print("A candlestick graph displays the relationship between the high, low, opening, and closing price of a stock. ")
    stock_data = yf.download(symbol, start= from_, end=to,)
    mpf.plot(stock_data, type='candle')




def plot_area(commerce_dates, open_close, symbol):
    print("The graph shows the aggregated close prices. \n")
    dates = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in commerce_dates]
    close_prices = [item[1] for item in open_close]

    percentage = (close_prices[-1] / close_prices[0]) * 100 - 100  #geeting the defference
    color = 'g' if close_prices[0] < close_prices[-1] else 'r'

    if close_prices[0] == close_prices[-1]:
      color = 'gray'
      percentage = 0

    percentage = f"Current Price: {close_prices[-1]},  {percentage:.2f}%"


    plt.figure(figsize=(12, 6), dpi=500)
    plt.plot(dates, close_prices, marker=',')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'{symbol}')
    plt.suptitle(percentage, fontsize=12, fontweight='bold', color=color)  # Adding the colored subtitle
    plt.fill_between(dates, close_prices, color=color)  # Fill area between open and close prices
    plt.tight_layout()
    plt.xticks(dates[::10], rotation=45)  # Adjust the step size as needed, here every 10th date is shown with a rotation of 45 degrees
    plt.tight_layout()
    plt.show()


In [ ]:

# Main function to interact with the user
def main():
    data_to_output_file = []

    # Load configurations from the text file and extract the key for request the api
    config = load_config('config.txt')

    symbol = config['symbol']
    key = config["key"]
    to = config["to"]
    from_ = config["from"]

    # update if neccesery
    symbol, from_, to = get_input(symbol, from_, to)



    # Fetch data from the stock API
    stock_data = get_stock_data(symbol, key, to , from_)


    # Check if the requested stock exists by the symbol that the user inserted
    if 'results' not in stock_data.keys():
      raise Exception("The Symbol doesn't exists. Please check your input")


    # total nums of day trades in the given date range
    number_commerce_days = stock_data["resultsCount"]

    # extract all dictioneries holding data per date of the given range
    stock_data_range_date = stock_data['results']
    print(stock_data_range_date)

    # Iterate over a the dictionaries and extract the dates to (yead-month-day format), open & close prices and high & low prices
    commerce_dates = [str(convert_unix_msec_to_datetime(diction['t']).strftime("%Y-%m-%d")) for diction in stock_data_range_date]
    open_close = [(diction['o'], diction['c']) for diction in stock_data_range_date]
    high_low = [(diction['h'], diction['l']) for diction in stock_data_range_date]



    # total move in this time period and taking 2 numbers after the period
    percentage = (open_close[-1][1] / open_close[0][1]) * 100 - 100
    percentage = "{:.2f}".format(percentage)

    # Add some data to the output file
    data_to_output_file.append("The Stock is: " + str(symbol))
    data_to_output_file.append("Choosen Dates: from: " + str(from_) + " to: " + str(to) + " total number of commere days are: " + str(number_commerce_days))
    data_to_output_file.append("The current price is: " + str(open_close[-1][1]) + " total move in this period: " + str(percentage)+"%")


    # maybe candle graph isnt counted as special feature

    if stock_data:
        print("********************************************************************************************************")
        print("Summery of the query: \n")
        for line in data_to_output_file:
          print(line)

        # wrtie the summrey into an output file
        write_output("output.txt", data_to_output_file)

    else:
        print("Failed to fetch stock data. Please check the symbol or try again later.")

    # Plotting Graphs
    plot_line(commerce_dates, open_close, symbol)
    print("\n")
    plot_area(commerce_dates, open_close, symbol)
    print("\n")
    plot_candle(symbol, str(from_), str(to)) # using special library
    print("\n")


main()
# maybe candle graph isnt counted as special feature


print config: {'key': 'ZMXwlS8PhYPKFNrEESjL90DpF4wf1faQ', 'symbol': 'NVDA', 'from': '2023-02-04', 'to': '2024-03-04'}
